In [1]:
import numpy as np
import matplotlib.pyplot as plt
from KMCLib import *
import time
#from KMCAnalysis_single_trajectory import *

# -----------------------------------------------------------------------------
# KMCLib version 2.0-a1
# Distributed under the GPLv3 license
# Copyright (C)  2012-2016  Mikael Leetmaa
# Developed by Mikael Leetmaa <leetmaa@kth.se>
#
# This program is distributed in the hope that it will be useful
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE and README files, and the source code, for details.
#
# You should have received a copy of the GNU General Public License version 3
# (GPLv3) along with this program. If not, see <http://www.gnu.org/licenses/>.
# -----------------------------------------------------------------------------



In [20]:
# parameters :
size = 100
# Define a squared unit cell of size 1x1x1.
cell_vectors = [[   1.000000e+00,   0.000000e+00,   0.000000e+00],
                [   0.000000e+00,   1.000000e+00,   0.000000e+00],
                [   0.000000e+00,   0.000000e+00,   1.000000e+00]]
# Define a unique point within the unit cell
basis_points = [[   0.500000e+00,   0.500000e+00,   0.500000e+00]]
unit_cell = KMCUnitCell(cell_vectors=cell_vectors,
                        basis_points=basis_points)
                        # Define a 1D lattice of size size
lattice = KMCLattice(unit_cell=unit_cell,
                     repetitions=(size,1,1),
                     periodic=(False, False, False))


In [3]:
# Generate the initial configuration array
types = ['S']*(size)# solvent everywhere
#types[size//2] = 'A'
#types[size//2:] = 'B'
types = ['A' if i <size//2 else 'B' for i in range(size)]
types = ['B','A','B','A','B','A','B','A','B','A']
#types[size//2+1] = 'C'
Nparticle = size
config = KMCConfiguration(lattice=lattice,
                          types=types,
                          possible_types=['S','A','B','C'])

In [4]:
lattice.unitCell().basis()[0][0]

0.5

In [25]:
from KMCLib.PluginInterfaces.KMCRateCalculatorPlugin import KMCRateCalculatorPlugin
Jaa,Jbb,Jcc = -10.,-10.,0.
Jab,Jac,Jbc = 1.,0.,0.

Vs = {'AA':Jaa,
        'BB':Jbb,
        'CC':Jcc,
        'AB':Jab,
        'BA':Jab,
        'AC':Jac,
        'CA':Jac,
        'BC':Jbc,
        'CB':Jbc,
        'AS':0.,
        'SA':0.,
        'BS':0.,
        'SB':0.,
        'CS':0.,
        'SC':0.,
        'SS':0.}

class CustomPlugin(KMCRateCalculatorPlugin):
    def rate(self,coords,types_before,types_after,rate_constant,process_number,global_coordinate):
        #print()
        #print(types_before)
        #print(types_after)
        #print([global_coordinate[0]+coords[i][0] for i in range(types_before.__len__())])
        #print(process_number)
        #print()
        neigh_B4 = {i:j for i,j in zip(coords[:,0],types_before)}
        neigh_AF = {i:j for i,j in zip(coords[:,0],types_after)}
        Eb4,Eaf = 0.,0.        
        if process_number<=11:
            # diffusive move
            Eb4 = Vs[neigh_B4.get(coords[0,0],'S')+neigh_B4.get(coords[0,0]-1,'S')] \
            +Vs[neigh_B4.get(coords[0,0],'S')+neigh_B4.get(coords[0,0]+1,'S')] \
            +Vs[neigh_B4.get(coords[1,0],'S')+neigh_B4.get(coords[1,0]-1,'S')]
            Eaf = Vs[neigh_AF.get(coords[0,0],'S')+neigh_AF.get(coords[0,0]-1,'S')] \
            +Vs[neigh_AF.get(coords[0,0],'S')+neigh_AF.get(coords[0,0]+1,'S')] \
            +Vs[neigh_AF.get(coords[1,0],'S')+neigh_AF.get(coords[1,0]-1,'S')]
        else:
            # chemical move
            Eb4 = Vs[neigh_B4.get(coords[0,0],'S')+neigh_B4.get(coords[0,0]-1,'S')] \
                + Vs[neigh_B4.get(coords[0,0],'S')+neigh_B4.get(coords[0,0]+1,'S')]
            Eaf = Vs[neigh_AF.get(coords[0,0],'S')+neigh_AF.get(coords[0,0]-1,'S')] \
                + Vs[neigh_AF.get(coords[0,0],'S')+neigh_AF.get(coords[0,0]+1,'S')]
        return rate_constant*np.exp((-Eaf+Eb4)/2)

    
    def cutoff(self):
        """ Determines the cutoff for this custom model """
        return 0.0

In [26]:
import Interacting_Processes as Process
# just diffusive process so far
interactions  = Process.make_interactions(1.,1.,ChemicalMove=True)
#from CustomRate import *
interactions.setRateCalculator(rate_calculator=CustomPlugin)

# Generate the KMC model to run.
model = KMCLatticeModel(configuration=config,
                        interactions=interactions)
control_parameters = KMCControlParameters(number_of_steps=10,
                                          dump_interval=1,
                                          analysis_interval=1,
                                          seed=None)

In [27]:
t0 = time.perf_counter()
model.run(control_parameters,trajectory_filename='test.py',
          trajectory_type='xyz')#,
          #analysis = MyAnalysis)
print('execution time : '+str(time.perf_counter()-t0))

 KMCLib: setting up the backend C++ object.

('B', 'B')
('A', 'B')
[0.5, 1.5]
13

-10.0
1.0

('B', 'B')
('C', 'B')
[0.5, 1.5]
16

-10.0
0.0

('B', 'B', 'B')
('A', 'B', 'B')
[1.5, 0.5, 2.5]
13

-20.0
2.0

('B', 'B', 'B')
('C', 'B', 'B')
[1.5, 0.5, 2.5]
16

-20.0
0.0

('B', 'B', 'B')
('A', 'B', 'B')
[2.5, 1.5, 3.5]
13

-20.0
2.0

('B', 'B', 'B')
('C', 'B', 'B')
[2.5, 1.5, 3.5]
16

-20.0
0.0

('B', 'B', 'B')
('A', 'B', 'B')
[3.5, 2.5, 4.5]
13

-20.0
2.0

('B', 'B', 'B')
('C', 'B', 'B')
[3.5, 2.5, 4.5]
16

-20.0
0.0

('B', 'B', 'A')
('A', 'B', 'A')
[4.5, 3.5, 5.5]
13

-9.0
-9.0

('B', 'B', 'A')
('C', 'B', 'A')
[4.5, 3.5, 5.5]
16

-9.0
0.0

('A', 'B', 'A', 'B', 'A')
('B', 'A', 'A', 'B', 'A')
[5.5, 4.5, 6.5, 3.5, 7.5]
6


('A', 'B', 'A')
('B', 'B', 'A')
[5.5, 4.5, 6.5]
12

-9.0
-9.0

('A', 'B', 'A')
('C', 'B', 'A')
[5.5, 4.5, 6.5]
14

-9.0
0.0

('A', 'A', 'A')
('B', 'A', 'A')
[6.5, 5.5, 7.5]
12

-20.0
2.0

('A', 'A', 'A')
('C', 'A', 'A')
[6.5, 5.5, 7.5]
14

-20.0
0.0

('A', 'A', 'A')
('B', '

In [19]:
from ReadXYZ import read_xyz_file

read_xyz_file('test.py')

STEP 10000 (TIME 147291404.2):
['B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'B']
